Let's go through SVD first.这里的SVD其实是将原始的评分矩阵分解成两个矩阵：这俩个子矩阵分别代表了用户的偏好程度和物品的成分程度，将这两个矩阵相乘即可得到用户对物品的评分矩阵。

The equation is:

$\hat{r}_{ui} = \mu + b_u + b_i + q_i^Tp_u$    这里前面三项均是bias


The loss function is

$\sum_{r_{ui} \in R_{train}} \left(r_{ui} - \hat{r}_{ui} \right)^2 + \lambda\left(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2\right)$



The update rule

$b_u \leftarrow b_u + \gamma (e_{ui} - \lambda b_u)$

$b_i \leftarrow b_i + \gamma (e_{ui} - \lambda b_i)$

$p_u \leftarrow p_u + \gamma (e_{ui} \cdot q_i - \lambda p_u)$

$q_i \leftarrow q_i + \gamma (e_{ui} \cdot p_u - \lambda q_i)$

We assume the input X, consists of three columns: the user_id, the item_id, and the rating.

In [5]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.random.randn(10, 4), columns=list('ABCD'))
print(df)
for index, row in df.iterrows():
    print(index)
    print(row['A'],row['B'])

          A         B         C         D
0  1.438929 -0.425226  0.233621 -1.245489
1  0.774340  1.584219  0.193919  1.051109
2 -0.716268 -0.320490  0.333734 -1.648085
3  0.268550 -0.761246 -0.734650 -0.548003
4  2.306601 -1.786420 -0.964326 -0.451474
5  1.213644  1.656571  1.255067 -0.138656
6 -1.550594 -1.263228 -0.346942 -0.376392
7 -1.239364 -0.200536  1.145938  0.521492
8 -0.117154  0.618509  0.065228 -0.435824
9 -0.502778 -0.726212 -1.088482 -0.047951
0
1.4389285830206415 -0.42522597264262857
1
0.7743403391728431 1.5842190598384063
2
-0.7162678223888921 -0.32049036963765054
3
0.2685499263601955 -0.7612460244841788
4
2.3066011200185446 -1.786420211766415
5
1.2136438937926033 1.6565707772802425
6
-1.5505940939209473 -1.2632282146253555
7
-1.2393644902409755 -0.2005356267028832
8
-0.11715382897869098 0.6185093991917099
9
-0.5027783123789128 -0.726211754003242


In [10]:
class Foo(object):
    def __init__(self):
        self.myattr = 0
        self.a = 1

    def bar(self):
        self.myattr += 1
        self.a += 1
        return self

f = Foo()
f.bar().bar().bar()
print(f.a)
print(f.myattr)

4
3


In [1]:
import numpy as np


class SVD:
    # What is mean?
    def __init__(self, learning_rate, regularized_rate, max_step, n_users, n_items, n_factors):
        self.learning_rate = learning_rate
        self.regularized_rate = regularized_rate
        self.max_step = max_step
        self.bu = np.zeros(n_users, np.double)
        self.bi = np.zeros(n_items, np.double)
        self.pu = np.zeros((n_users, n_factors), np.double)
        self.qi = np.zeros((n_items, n_factors), np.double)
        self.mean = 0   # 就是公式中的u,
        
    def get_pred_value(u, i):
        return self.mean + self.bu[u] + self.bi[i] + np.dot(self.pu[u], self.qi[i])
        
    # There is a problem here, can anyone find it?
    def fit(self, X, y):
        for index, row in X.iterrows():
            u, i, r = row['user_id'], row['item_id'], row['rating']
            error = r - self.get_pred_value(u, i)
            self.bu[u] += self.learning_rate * (err - self.regularized_rate * self.bu[u])
            self.bi[i] += self.learning_rate * (err - self.regularized_rate * self.bi[i])
            tmp = self.pu[u]
            self.pu[u] += self.learning_rate * (err * self.qi[i] - self.regularized_rate * self.pu[u])
            self.qi[i] += self.learning_rate * (err * tmp - self.regularized_rate * self.qi[i])
            if index == self.max_step:
                break
        return self
            
    # What if the one to predict is not inside our knowledge?
    def transform(self, X):
        result = [0] * len(X)
        for index, row in X.iterrows():
            u, i, r = row['user_id'], row['item_id'], row['rating']
            result[index] = self.get_pred_value(u, i)
        return result

Here comes SVD++.相比SVD只在user_embedding矩阵上加了一个该用户u做出了评价的item的评分之和并做了标准化处理

$\hat{r}_{ui} = \mu + b_u + b_i + q_i^T\left(p_u + |I_u|^{-\frac{1}{2}} \sum_{j \in I_u}y_j\right)$   这里I_u代表的是用户u做了评价的item list

Remember we also modified the loss function:

$\sum_{r_{ui} \in R_{train}} \left(r_{ui} - \hat{r}_{ui} \right)^2 + \lambda\left(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2 + ||y_j||^2 \right)$

And the update rule:

$q_i \leftarrow q_i + \gamma (e_{ui} \cdot (p_u + |I_u|^{-\frac{1}{2}} \sum_{j \in I_u}y_j) - \lambda q_i)$

In [2]:
class SVDPP:
    def __init__(self, learning_rate, regularized_rate, max_step, n_users, n_items, n_factors, ur_dict):
        
        self.learning_rate = learning_rate
        self.regularized_rate = regularized_rate
        self.max_step = max_step
        self.bu = np.zeros(n_users, np.double)
        self.bi = np.zeros(n_items, np.double)
        self.pu = np.zeros((n_users, n_factors), np.double)
        self.qi = np.zeros((n_items, n_factors), np.double)
        self.yj = np.zeros((n_items, n_factors), np.double)
        self.ur_dict = ur_dict
        self.mean = 0
        
    def get_pred_value(u, i, u_impl_fdb):
        return self.mean + self.bu[u] + self.bi[i] + np.dot(self.pu[u] + u_impl_fdb, self.qi[i])
        
    def fit(self, X, y):
        for index, row in X.iterrows():
            u, i, r = row['user_id'], row['item_id'], row['rating']
            
            # suppose we can get a user's rating related to all items he rated
            # ur_dict : {user_id:{item_id:打分,...},...}
            Iu = [j for (j, _) in self.ur_dict[u]]
            sqrt_Iu = np.sqrt(len(Iu))
            u_impl_fdb = np.zeros(self.n_factors, np.double)
            for j in Iu:
                u_impl_fdb += self.yj[j] / sqrt_Iu
            
            error = r - self.get_pred_value(u, i, u_impl_fdb)
            self.bu[u] += self.learning_rate * (err - self.regularized_rate * self.bu[u])
            self.bi[i] += self.learning_rate * (err - self.regularized_rate * self.bi[i])
            self.pu[u] += self.learning_rate * (err * self.qi[i] - self.regularized_rate * self.pu[u])
            self.qi[i] += self.learning_rate * (err * (self.pu[u] + u_impl_fdb) - self.regularized_rate * self.qi[i])
            
            for j in Iu:
                self.yj[j] += self.learning_rate * (err * self.qi[i] / sqrt_Iu - self.regularized_rate * self.yj[j])
                
            if index == self.max_step:
                break
        return self
            
    def transform(self, X):
        result = [0] * len(X)
        for index, row in X.iterrows():
            u, i, r = row['user_id'], row['item_id'], row['rating']
            result[index] = self.get_pred_value(u, i)
        return result

Now we consider a final case, NMF.

NMF is a collaborative filtering algorithm based on Non-negative Matrix Factorization.

$ p_{uf} \leftarrow p_{uf} \cdot \frac{\sum_{i \in I_u} q_{if} \cdot r_{ui}}{\sum_{i \in I_u} q_{if} \cdot \hat{r_{ui}} + \lambda_u |I_u| p_{uf}} $
        
$ q_{if} \leftarrow q_{if} \cdot \frac{\sum_{u \in U_i} p_{uf} \cdot r_{ui}}{\sum_{u \in U_i} p_{uf} \cdot \hat{r_{ui}} + \lambda_i |U_i| q_{if}} $

In [ ]:
class NMF:
    def __init__(self, learning_rate, regularized_rate, max_step, n_users, n_items, n_factors, ur_dict, ir_dict):
        self.learning_rate = learning_rate
        self.regularized_rate = regularized_rate
        self.max_step = max_step
        self.bu = np.zeros(n_users, np.double)
        self.bi = np.zeros(n_items, np.double)
        self.pu = np.zeros((n_users, n_factors), np.double)
        self.qi = np.zeros((n_items, n_factors), np.double)
        self.n_users = n_users
        self.n_items = n_items
        self.n_factors = n_factors
        self.ur_dict = ur_dict
        self.ir_dict = ir_dict
        self.mean = 0
        
    def get_pred_value(u, i):
        return self.mean + self.bu[u] + self.bi[i] + np.dot(self.pu[u], self.qi[i])
        
    def fit(self, X, y):
        user_num = np.zeros((self.n_users, self.n_factors))
        user_denom = np.zeros((self.n_users, self.n_factors))
        item_num = np.zeros((self.n_items, self.n_factors))
        item_denom = np.zeros((self.n_items, self.n_factors))
        for index, row in X.iterrows():
            u, i, r = row['user_id'], row['item_id'], row['rating']
        
            est = self.get_pred_value(u, i)
            error = r - self.get_pred_value(u, i)
            self.bu[u] += self.learning_rate * (err - self.regularized_rate * self.bu[u])
            self.bi[i] += self.learning_rate * (err - self.regularized_rate * self.bi[i])
            
            user_num[u] += self.qi[i] *r
            user_denom[u] += self.qi[i] * est
            item_num[i] += self.pu[u] * r
            item_denom[i] += self.pu[u] * est
            
            if index == self.max_step:
                break

        for u in range(self.n_users):
            ratings = self.ur[u]
            user_denom[u] += ratings * self.learning_rate * self.pu[u]
            self.pu[u] *= user_num[u] / user_denom[u]
         
        for i in range(self.n_items):
            ratings = self.ir[i]
            item_denom[i] += ratings * self.learning_rate * self.qi[i]
            self.qi[i] *= item_num[i] / item_denom[i]

        return self
            
    def transform(self, X):
        result = [0] * len(X)
        for index, row in X.iterrows():
            u, i, r = row['user_id'], row['item_id'], row['rating']
            result[index] = self.get_pred_value(u, i)
        return result

LinUCB

$\beta = (X^TX+I)^{-1}(X^Ty)$

$A = X^TX+I$

$b = X^Ty$

$p = \beta^TX + \alpha\sqrt{X^TA^{-1}X}$

($\alpha = 1 + \sqrt{\ln(2/\delta)/2}$)

$A \leftarrow A + xx^T$

$B \leftarrow B + rx^T$

In [ ]:
DCN

Cross Network